Things to do:
+ scoring - sum the two parts
+ Club column - fill nan with None
+ Drop: ID, Name, Nationality, BP, Position, team & contract, Growth, Joined, Loan Date End (?), Contract, 

+ Bins: Joined - 6 months

+ Probably not relevant: Value, Wage, Release Clause

+ Columns to drop:
Joined
Loan Date End

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

%matplotlib inline

# Import dataset
dataset = pd.read_csv('fifa21_train.csv')
dataset.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,0,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,13,"Jul 1, 2016",NaN,€275K,€4K,€694K,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,8,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [2]:
# Function to convert K and M to normal numbering
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    return float(x)

# Character replacing function
def remove_char(x,y, z=''):
    return x.apply(lambda d: d.replace(y, z))


# function to split the columns and sum the values
def split_columns(data):
    n1 = data.str.split('+',n=1,expand=True)[0]
    n2 = data.str.split('+',n=1,expand=True)[1]
    n1 = pd.to_numeric(n1)
    n2 = pd.to_numeric(n2)

    return n1 + n2

# Functions to apply python snake column names
def std_data(x, y='_'):
    x.columns = [e.lower().replace(' ','_') for e in x]
    return x

# Data = working dataset, drp = columns to drop
def fill_na(data, drp):
    
    #dropping unwanted columns
    data = data.drop(columns=drp, axis=1)
    data = data[data['composure'].isna() == False]
    
    #fill Nan's with values
    data['position'].fillna(value=data['bp'], inplace=True)
    data['club'].fillna(value='None', inplace=True)
    data = data.reset_index(drop=True)
    
    return data

# Variables to change parameters of cleaning
columns_to_drop = ['loan_date_end', 'id', 'name', 'team_&_contract', 'growth', 'joined', 'contract']

cleandata = std_data(dataset)
cleandata.rename(columns = {'height':'height_cm', 'weight':'weight_kg', 'value':'value_euro', 'wage':'wage_euro', 'release_clause':'release_clause_euro'}, inplace = True)
cleandata = fill_na(cleandata, columns_to_drop)
drop=["age","nationality","club","height_cm","weight_kg","foot","value_euro","wage_euro","release_clause_euro","attacking","skill","movement","power","mentality","defending","goalkeeping","total_stats","base_stats","ls","st","rs","lw","lf","cf","rf","rw","lam","cam","ram","lm","lcm","cm","rcm","rm","lwb","ldm","cdm","rdm","rwb","lb","lcb","cb","rcb","rb","gk"]

cleandata = cleandata.drop(columns=drop, axis=1)

cleandata.head()

,bp,position,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ova
0,CM,CM CDM,54,47,43,70,44.0,61,44.0,55,63,63,64,73,61.0,66,82.0,62,73.0,71,55,45,54,52.0,62.0,68.0,54,54.0,49,56,43.0,7,12,14,9,6,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,64
1,ST,ST LW LM,66,79,76,68,76.0,83,78.0,72,63,79,83,83,88.0,75,75.0,74,81.0,75,74,68,54,33.0,78.0,72.0,76,70.0,35,20,22.0,11,7,14,7,16,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77
2,CAM,CAM CF,73,76,34,78,75.0,85,89.0,91,74,85,84,76,93.0,78,93.0,79,34.0,75,42,78,75,26.0,80.0,78.0,73,82.0,23,29,28.0,6,3,6,3,3,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,80
3,CDM,CDM CM,44,42,58,62,36.0,54,41.0,46,57,61,54,59,59.0,55,55.0,57,60.0,64,58,38,61,57.0,31.0,54.0,54,48.0,55,58,55.0,8,9,6,7,12,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,59
4,CDM,CDM CM,49,37,61,68,34.0,64,44.0,45,61,66,66,66,61.0,62,69.0,61,34.0,81,61,43,66,60.0,55.0,64.0,49,58.0,58,61,66.0,8,9,15,5,15,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,65


In [3]:
# cleandata = cleandata.drop(columns=['nationality', 'club', 'bp', 'position', 'foot', 'a/w', 'd/w', 'contract'])

In [4]:
cleandata.head(50)

,bp,position,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ova
0,CM,CM CDM,54,47,43,70,44.0,61,44.0,55,63,63,64,73,61.0,66,82.0,62,73.0,71,55,45,54,52.0,62.0,68.0,54,54.0,49,56,43.0,7,12,14,9,6,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,64
1,ST,ST LW LM,66,79,76,68,76.0,83,78.0,72,63,79,83,83,88.0,75,75.0,74,81.0,75,74,68,54,33.0,78.0,72.0,76,70.0,35,20,22.0,11,7,14,7,16,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77
2,CAM,CAM CF,73,76,34,78,75.0,85,89.0,91,74,85,84,76,93.0,78,93.0,79,34.0,75,42,78,75,26.0,80.0,78.0,73,82.0,23,29,28.0,6,3,6,3,3,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,80
3,CDM,CDM CM,44,42,58,62,36.0,54,41.0,46,57,61,54,59,59.0,55,55.0,57,60.0,64,58,38,61,57.0,31.0,54.0,54,48.0,55,58,55.0,8,9,6,7,12,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,59
4,CDM,CDM CM,49,37,61,68,34.0,64,44.0,45,61,66,66,66,61.0,62,69.0,61,34.0,81,61,43,66,60.0,55.0,64.0,49,58.0,58,61,66.0,8,9,15,5,15,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,65
5,LM,LM LW RW,69,64,64,72,59.0,71,62.0,64,64,72,73,74,75.0,69,67.0,65,63.0,70,64,59,56,38.0,68.0,74.0,64,71.0,31,34,38.0,16,10,15,8,16,4 ★,3★,High,Low,1 ★,74,63,70,71,37,64,6,71
6,RB,RB LB,69,55,68,74,59.0,72,69.0,35,66,74,68,67,77.0,74,75.0,69,67.0,75,62,56,74,77.0,63.0,59.0,45,78.0,78,80,77.0,6,7,7,12,7,5 ★,3★,Medium,High,2 ★,67,58,67,73,77,68,49,76
7,LM,CAM,58,56,42,68,38.0,60,55.0,39,63,56,62,62,67.0,60,72.0,50,55.0,57,51,27,45,33.0,63.0,65.0,39,50.0,41,49,39.0,6,14,12,10,9,2 ★,2★,Medium,Medium,1 ★,62,48,63,60,42,52,4,60
8,CB,CDM CB,54,46,70,63,44.0,56,43.0,57,62,63,33,35,34.0,64,34.0,54,86.0,57,90,56,62,69.0,62.0,52.0,49,63.0,58,69,65.0,15,6,7,9,12,2 ★,2★,Medium,Medium,1 ★,34,50,58,55,65,76,4,65
9,CB,LB CB,73,56,65,63,49.0,63,34.0,48,60,65,53,55,66.0,63,62.0,70,74.0,62,70,70,80,67.0,61.0,65.0,56,63.0,68,70,67.0,6,10,16,11,11,3 ★,3★,Medium,Medium,1 ★,54,62,63,64,68,70,3,67


In [5]:
# currency_values = ['value_euro', 'wage_euro', 'release_clause_euro']

# #Characters to replace
# for e in currency_values:
#     cleandata[e] = remove_char(cleandata[e],'€')
#     cleandata[e] = cleandata[e].apply(value_to_float)

# cleandata['hits'] = cleandata['hits'].apply(value_to_float)
# cleandata.head()

In [6]:
# Converting height to cm
# cleandata['weight_kg'] = remove_char(cleandata['weight_kg'],'lbs')
# cleandata['weight_kg'] = cleandata['weight_kg'].astype(float)*0.4532
# cleandata['height_cm'] = remove_char(cleandata['height_cm'],'"')
# cleandata['height_cm'] = ((cleandata.height_cm.str.split("'").str[0].astype(int) * 12) + (cleandata.height_cm.str.split("'").str[1].astype(int)))*2.54

In [7]:
# cleandata['weight_lbs'] = remove_char(cleandata['weight'],'lbs')
to_drop = ['w/f', 'sm', 'ir']

for e in to_drop:
    cleandata[e] = remove_char(cleandata[e],'★')
    cleandata[e] = cleandata[e].astype(int)


In [8]:
# plus_items = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

# for e in plus_items:
#     for i in cleandata.columns:
#         #test if values of i and e are equal
#         if i == e:
#             #calling split function to make the sum
#             cleandata[e] = split_columns(cleandata[e])

In [9]:
cleandata.head(50)

,bp,position,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ova
0,CM,CM CDM,54,47,43,70,44.0,61,44.0,55,63,63,64,73,61.0,66,82.0,62,73.0,71,55,45,54,52.0,62.0,68.0,54,54.0,49,56,43.0,7,12,14,9,6,4,2,High,Medium,1,69,51,63,63,51,60,3,64
1,ST,ST LW LM,66,79,76,68,76.0,83,78.0,72,63,79,83,83,88.0,75,75.0,74,81.0,75,74,68,54,33.0,78.0,72.0,76,70.0,35,20,22.0,11,7,14,7,16,3,4,High,Low,2,83,75,68,82,33,71,44,77
2,CAM,CAM CF,73,76,34,78,75.0,85,89.0,91,74,85,84,76,93.0,78,93.0,79,34.0,75,42,78,75,26.0,80.0,78.0,73,82.0,23,29,28.0,6,3,6,3,3,4,4,High,Medium,2,80,77,78,86,27,56,73,80
3,CDM,CDM CM,44,42,58,62,36.0,54,41.0,46,57,61,54,59,59.0,55,55.0,57,60.0,64,58,38,61,57.0,31.0,54.0,54,48.0,55,58,55.0,8,9,6,7,12,2,2,Medium,Medium,1,57,44,54,57,57,60,7,59
4,CDM,CDM CM,49,37,61,68,34.0,64,44.0,45,61,66,66,66,61.0,62,69.0,61,34.0,81,61,43,66,60.0,55.0,64.0,49,58.0,58,61,66.0,8,9,15,5,15,2,3,Low,Medium,1,66,44,60,64,60,66,4,65
5,LM,LM LW RW,69,64,64,72,59.0,71,62.0,64,64,72,73,74,75.0,69,67.0,65,63.0,70,64,59,56,38.0,68.0,74.0,64,71.0,31,34,38.0,16,10,15,8,16,4,3,High,Low,1,74,63,70,71,37,64,6,71
6,RB,RB LB,69,55,68,74,59.0,72,69.0,35,66,74,68,67,77.0,74,75.0,69,67.0,75,62,56,74,77.0,63.0,59.0,45,78.0,78,80,77.0,6,7,7,12,7,5,3,Medium,High,2,67,58,67,73,77,68,49,76
7,LM,CAM,58,56,42,68,38.0,60,55.0,39,63,56,62,62,67.0,60,72.0,50,55.0,57,51,27,45,33.0,63.0,65.0,39,50.0,41,49,39.0,6,14,12,10,9,2,2,Medium,Medium,1,62,48,63,60,42,52,4,60
8,CB,CDM CB,54,46,70,63,44.0,56,43.0,57,62,63,33,35,34.0,64,34.0,54,86.0,57,90,56,62,69.0,62.0,52.0,49,63.0,58,69,65.0,15,6,7,9,12,2,2,Medium,Medium,1,34,50,58,55,65,76,4,65
9,CB,LB CB,73,56,65,63,49.0,63,34.0,48,60,65,53,55,66.0,63,62.0,70,74.0,62,70,70,80,67.0,61.0,65.0,56,63.0,68,70,67.0,6,10,16,11,11,3,3,Medium,Medium,1,54,62,63,64,68,70,3,67


In [10]:
# Getting numerical and categorical data separated

numerical = cleandata.select_dtypes(np.number)
categorical = cleandata.select_dtypes('object')


In [11]:
cleandata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11422 entries, 0 to 11421
Data columns (total 49 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bp                11422 non-null  object 
 1   position          11422 non-null  object 
 2   crossing          11422 non-null  int64  
 3   finishing         11422 non-null  int64  
 4   heading_accuracy  11422 non-null  int64  
 5   short_passing     11422 non-null  int64  
 6   volleys           11422 non-null  float64
 7   dribbling         11422 non-null  int64  
 8   curve             11422 non-null  float64
 9   fk_accuracy       11422 non-null  int64  
 10  long_passing      11422 non-null  int64  
 11  ball_control      11422 non-null  int64  
 12  acceleration      11422 non-null  int64  
 13  sprint_speed      11422 non-null  int64  
 14  agility           11422 non-null  float64
 15  reactions         11422 non-null  int64  
 16  balance           11422 non-null  float6

In [12]:
categorical.head()

,bp,position,a/w,d/w,hits
0,CM,CM CDM,High,Medium,3
1,ST,ST LW LM,High,Low,44
2,CAM,CAM CF,High,Medium,73
3,CDM,CDM CM,Medium,Medium,7
4,CDM,CDM CM,Low,Medium,4


In [13]:
numerical.head()

,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,w/f,sm,ir,pac,sho,pas,dri,def,phy,ova
0,54,47,43,70,44.0,61,44.0,55,63,63,64,73,61.0,66,82.0,62,73.0,71,55,45,54,52.0,62.0,68.0,54,54.0,49,56,43.0,7,12,14,9,6,4,2,1,69,51,63,63,51,60,64
1,66,79,76,68,76.0,83,78.0,72,63,79,83,83,88.0,75,75.0,74,81.0,75,74,68,54,33.0,78.0,72.0,76,70.0,35,20,22.0,11,7,14,7,16,3,4,2,83,75,68,82,33,71,77
2,73,76,34,78,75.0,85,89.0,91,74,85,84,76,93.0,78,93.0,79,34.0,75,42,78,75,26.0,80.0,78.0,73,82.0,23,29,28.0,6,3,6,3,3,4,4,2,80,77,78,86,27,56,80
3,44,42,58,62,36.0,54,41.0,46,57,61,54,59,59.0,55,55.0,57,60.0,64,58,38,61,57.0,31.0,54.0,54,48.0,55,58,55.0,8,9,6,7,12,2,2,1,57,44,54,57,57,60,59
4,49,37,61,68,34.0,64,44.0,45,61,66,66,66,61.0,62,69.0,61,34.0,81,61,43,66,60.0,55.0,64.0,49,58.0,58,61,66.0,8,9,15,5,15,2,3,1,66,44,60,64,60,66,65


In [14]:
def cat_encode(data, exclude=''):
    
    #Categorical encoder Label + onehot
    #If no fields to label encode are provided only the one hot encoder is executed
    if exclude == '':
        
        encoder = OneHotEncoder(drop='first').fit(data)
        encoded = encoder.transform(data).toarray()
        cols = encoder.get_feature_names(input_features=data.columns)
        onehot_encoded = pd.DataFrame(encoded, columns=cols)
        return pd.concat([X_normalized, onehot_encoded], axis=1) 
        
    else:
        
        #Label Encoder
        le = LabelEncoder()
        label_encoded = data[exclude]
        label_encoded[exclude] = label_encoded[exclude].apply(le.fit_transform)
        
        #One hot encoder
        tmp = data.drop(columns=exclude, axis=1)
        encoder = OneHotEncoder(drop='first').fit(tmp)
        encoded = encoder.transform(tmp).toarray()
        cols = encoder.get_feature_names(input_features=tmp.columns)
        onehot_encoded = pd.DataFrame(encoded, columns=cols)
        
        return pd.concat([onehot_encoded, label_encoded], axis=1) 
    
# categorical = cat_encode(categorical, ['a/w', 'd/w'])
categorical.head()

,bp,position,a/w,d/w,hits
0,CM,CM CDM,High,Medium,3
1,ST,ST LW LM,High,Low,44
2,CAM,CAM CF,High,Medium,73
3,CDM,CDM CM,Medium,Medium,7
4,CDM,CDM CM,Low,Medium,4


In [15]:
correlations_matrix = numerical.corr()
# sns.set(rc = {'figure.figsize':(40,40)})
# sns.heatmap(correlations_matrix)
# plt.show()
display(correlations_matrix)

,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,w/f,sm,ir,pac,sho,pas,dri,def,phy,ova
crossing,1.000000,0.646256,0.437497,0.799548,0.675034,0.855271,0.833267,0.751567,0.740166,0.835232,0.640668,0.607835,0.692555,0.372578,0.623959,0.527896,0.094863,0.643401,-0.056805,0.732963,0.460590,0.403521,0.780385,0.683351,0.628233,0.598724,0.424551,0.409460,0.389442,-0.648840,-0.647743,-0.644700,-0.647977,-0.652027,0.281059,0.713070,0.180405,0.309447,0.200691,0.500359,0.443280,0.230739,0.001912,0.390126
finishing,0.646256,1.000000,0.455249,0.651879,0.892031,0.822037,0.762645,0.697396,0.488899,0.785340,0.574356,0.558185,0.630683,0.329221,0.514262,0.725919,0.066202,0.482171,0.003983,0.885144,0.245361,-0.067406,0.898995,0.695267,0.843914,0.568119,-0.025338,-0.068166,-0.110564,-0.580238,-0.579193,-0.574286,-0.576953,-0.577976,0.364168,0.732395,0.161021,0.286862,0.605760,0.320397,0.468006,-0.261554,-0.057035,0.306170
heading_accuracy,0.437497,0.455249,1.000000,0.632043,0.490900,0.535227,0.415693,0.367583,0.482946,0.641047,0.261362,0.322725,0.203942,0.330303,0.122254,0.365009,0.403066,0.581940,0.504851,0.481864,0.689388,0.503860,0.519530,0.241427,0.537302,0.523465,0.541604,0.516525,0.489470,-0.716163,-0.714990,-0.710915,-0.709594,-0.715512,0.164557,0.375158,0.164350,-0.167444,-0.120863,-0.096552,-0.148947,0.378944,0.405321,0.300564
short_passing,0.799548,0.651879,0.632043,1.000000,0.684156,0.841014,0.766269,0.719659,0.887742,0.914049,0.516154,0.503140,0.584710,0.490542,0.521521,0.579362,0.179854,0.698027,0.140094,0.752718,0.618536,0.531787,0.752401,0.707138,0.664138,0.727105,0.558200,0.536093,0.500800,-0.719138,-0.717723,-0.714305,-0.714360,-0.721072,0.306580,0.684787,0.233754,0.101767,0.149695,0.462299,0.346886,0.365485,0.172567,0.492056
volleys,0.675034,0.892031,0.490900,0.684156,1.000000,0.803731,0.805333,0.746409,0.542818,0.787478,0.526609,0.506744,0.603646,0.385506,0.499346,0.752385,0.098129,0.483077,0.039570,0.873797,0.319703,0.026697,0.856679,0.693264,0.842998,0.625996,0.062031,0.019350,-0.020316,-0.575849,-0.575401,-0.569804,-0.572327,-0.575006,0.368453,0.733689,0.241052,0.224359,0.547850,0.369366,0.454587,-0.156439,-0.003379,0.360874
dribbling,0.855271,0.822037,0.535227,0.841014,0.803731,1.000000,0.845919,0.746695,0.706802,0.939990,0.717692,0.691067,0.755854,0.355475,0.660373,0.615415,0.106775,0.664902,-0.038938,0.842532,0.443055,0.266807,0.900869,0.725141,0.762551,0.638322,0.307657,0.282351,0.252909,-0.754740,-0.753293,-0.749392,-0.751628,-0.754900,0.340470,0.807109,0.151087,0.334021,0.280813,0.359759,0.490360,0.056550,-0.042258,0.351380
curve,0.833267,0.762645,0.415693,0.766269,0.805333,0.845919,1.000000,0.854783,0.694798,0.826546,0.574932,0.534388,0.673910,0.401466,0.591332,0.684042,0.074015,0.558549,-0.049054,0.837144,0.391613,0.237620,0.818279,0.748669,0.747572,0.650195,0.261062,0.234908,0.203882,-0.593173,-0.592690,-0.587827,-0.592043,-0.595811,0.333971,0.758209,0.226600,0.260595,0.405982,0.519742,0.504755,0.062498,-0.019155,0.401497
fk_accuracy,0.751567,0.697396,0.367583,0.719659,0.746409,0.746695,0.854783,1.000000,0.689473,0.746451,0.454352,0.411852,0.575695,0.374719,0.518103,0.661530,0.037055,0.493284,-0.034158,0.806442,0.375503,0.249961,0.726370,0.716029,0.728781,0.608777,0.259313,0.241513,0.206727,-0.531264,-0.531681,-0.526591,-0.530785,-0.533487,0.324285,0.683712,0.224469,0.148717,0.399518,0.524140,0.437992,0.087268,-0.007719,0.369877
long_passing,0.740166,0.488899,0.482946,0.887742,0.542818,0.706802,0.694798,0.689473,1.000000,0.774891,0.381735,0.359259,0.491156,0.467141,0.451579,0.523686,0.129285,0.611588,0.109239,0.650318,0.585803,0.592713,0.595267,0.700910,0.519029,0.678792,0.596877,0.588493,0.563614

In [16]:
def executelr(data, target, exclude='', c_drop=''):

    #Data split
    y = data[target]
    X = data.drop([target], axis=1)

    #Columns drop if necessary
    if c_drop!='':
        X = X.drop(columns=c_drop, axis=1)

    #MinMax normalizing
    X_num = X.select_dtypes(include = np.number)
    X_cat = X.select_dtypes(include = object)

    transformer = MinMaxScaler().fit(X_num)
    X_normalized = transformer.transform(X_num)
    X_normalized = pd.DataFrame(X_normalized, columns=X_num.columns)

    
    #Categorical encoder Label + onehot
    #If no fields to label encode are provided only the one hot encoder is executed
    if exclude == '':
        
        encoder = OneHotEncoder(drop='first').fit(X_cat)
        encoded = encoder.transform(X_cat).toarray()
        cols = encoder.get_feature_names(input_features=X_cat.columns)
        onehot_encoded = pd.DataFrame(encoded, columns=cols)
        #Concat normalized min max + onehot + label encoded
        X = pd.concat([X_normalized, onehot_encoded], axis=1) 
        
    else:
        
        #Label Encoder
        le = LabelEncoder()
        label_encoded = X_cat[exclude]
        label_encoded[exclude] = label_encoded[exclude].apply(le.fit_transform)
        
        #One hot encoder
        tmp = X_cat.drop(columns=exclude, axis=1)
        encoder = OneHotEncoder(drop='first').fit(tmp)
        encoded = encoder.transform(tmp).toarray()
        cols = encoder.get_feature_names(input_features=tmp.columns)
        onehot_encoded = pd.DataFrame(encoded, columns=cols)
        
        #Concat normalized min max + onehot + label encoded
        X = pd.concat([X_normalized, onehot_encoded, label_encoded], axis=1) 

    #train test Splits
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

    #linear Regression
    lm = linear_model.LinearRegression()
    lm.fit(X_train,y_train)

    predictions = lm.predict(X_train)
    predictions_test = lm.predict(X_test)

    # results Validation
    print('\n\nLinear Regression Performance Results\n')
    print('  R2 SCORE: Train', round(r2_score(y_train, predictions),3), '     | Test', round(r2_score(y_test, predictions_test), 3))
    print(' MSE SCORE: Train', round(mean_squared_error(y_train,predictions),3), ' | Test', round(mean_squared_error(y_test,predictions_test), 3))
    print('RMSE SCORE: Train', round(np.sqrt(mean_squared_error(y_train,predictions)),3), '   | Test', round(np.sqrt(mean_squared_error(y_test,predictions_test)),3))
    print(' MAE SCORE: Train', round(mean_absolute_error(y_train, predictions),3), '    | Test', round(mean_absolute_error(y_test, predictions_test), 3))

    print('\n\nTraining Values')
    display(y_train[:5])

    print('Prediction Values')
    display(predictions[:5])
    return lm

lm = executelr(cleandata,'ova')

C:\Users\luana\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)




Linear Regression Performance Results

  R2 SCORE: Train 0.94      | Test -1.037056647281548e+20
 MSE SCORE: Train 2.83  | Test 4.857899693719028e+21
RMSE SCORE: Train 1.682    | Test 69698634805.275
 MAE SCORE: Train 1.283     | Test 6670420475.405


Training Values


2249    60
4952    80
6884    82
6008    71
6499    65
Name: ova, dtype: int64

Prediction Values


array([60.88661959, 79.99975719, 83.05897645, 69.9526299 , 65.70551374])

In [21]:
# Import new dataset to predict 
dataset_val = pd.read_csv('fifa21_validate.csv')
dataset_val.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"FK Austria Wien Jun 30, 2021 On Loan","6'2""",194lbs,Right,7,"Feb 8, 2018","Jun 30, 2021",€975K,€5K,€0,"Jun 30, 2021 On Loan",230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2 ★,2★,Low,High,1 ★,64,30,50,50,66,74,34,48+2,48+2,48+2,48+0,47+0,47+0,47+0,48+0,48+2,48+2,48+2,50+2,51+2,51+2,51+2,50+2,59+2,61+2,61+2,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2,67
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,Heart of Midlothian 2020 ~ 2020,"5'8""",154lbs,Right,5,"Jan 20, 2020",NaN,€1.2M,€3K,€2.2M,2020 ~ 2020,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4 ★,3★,High,Low,1 ★,74,67,62,74,24,57,12,64+2,64+2,64+2,68+0,68+0,68+0,68+0,68+0,68+2,68+2,68+2,67+2,61+2,61+2,61+2,67+2,49+2,47+2,47+2,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2,68
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,Real Salt Lake 2018 ~ 2020,"6'2""",176lbs,Right,17,"Nov 28, 2018",NaN,€120K,€500,€249K,2018 ~ 2020,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2 ★,1★,Medium,Medium,1 ★,56,52,53,55,26,53,3,18+2,18+2,18+2,15+0,17+0,17+0,17+0,15+0,17+2,17+2,17+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2,54
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,Leeds United 2020 ~ 2022,"6'3""",170lbs,Right,23,"Jan 10, 2020",NaN,€160K,€500,€464K,2020 ~ 2022,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3 ★,2★,Medium,Medium,1 ★,58,34,47,52,53,59,6,46+2,46+2,46+2,47+0,46+0,46+0,46+0,47+0,47+2,47+2,47+2,49+2,49+2,49+2,49+2,49+2,53+2,54+2,54+2,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2,55
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,1. FSV Mainz 05 2019 ~ 2023,"6'2""",170lbs,Right,5,"Jul 1, 2019",NaN,€2.3M,€13K,€4.3M,2019 ~ 2023,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4 ★,2★,Medium,Medium,1 ★,67,62,68,70,69,71,45,63+2,63+2,63+2,66+0,66+0,66+0,66+0,66+0,68+2,68+2,68+2,67+2,70+2,70+2,70+2,67+2,70+2,72+2,72+2,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2,70


In [23]:
# Function to convert K and M to normal numbering
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    return float(x)

# Character replacing function
def remove_char(x,y, z=''):
    return x.apply(lambda d: d.replace(y, z))


# function to split the columns and sum the values
def split_columns(data):
    n1 = data.str.split('+',n=1,expand=True)[0]
    n2 = data.str.split('+',n=1,expand=True)[1]
    n1 = pd.to_numeric(n1)
    n2 = pd.to_numeric(n2)

    return n1 + n2

# Functions to apply python snake column names
def std_data(x, y='_'):
    x.columns = [e.lower().replace(' ','_') for e in x]
    return x

# Data = working dataset, drp = columns to drop
def fill_na(data, drp):
    
    #dropping unwanted columns
    data = data.drop(columns=drp, axis=1)
    data = data[data['composure'].isna() == False]
    
    #fill Nan's with values
    data['position'].fillna(value=data['bp'], inplace=True)
    data['club'].fillna(value='None', inplace=True)
    data = data.reset_index(drop=True)
    
    return data

# Variables to change parameters of cleaning
columns_to_drop = ['loan_date_end', 'id', 'name', 'team_&_contract', 'growth', 'joined', 'contract']

cleandata_val = std_data(dataset_val)
cleandata_val.rename(columns = {'height':'height_cm', 'weight':'weight_kg', 'value':'value_euro', 'wage':'wage_euro', 'release_clause':'release_clause_euro'}, inplace = True)
cleandata_val = fill_na(cleandata_val, columns_to_drop)
drop=["age","nationality","club","height_cm","weight_kg","foot","value_euro","wage_euro","release_clause_euro","attacking","skill","movement","power","mentality","defending","goalkeeping","total_stats","base_stats","ls","st","rs","lw","lf","cf","rf","rw","lam","cam","ram","lm","lcm","cm","rcm","rm","lwb","ldm","cdm","rdm","rwb","lb","lcb","cb","rcb","rb","gk"]

cleandata_val = cleandata_val.drop(columns=drop, axis=1)

cleandata_val.head()

,bp,position,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ova
0,CB,CB,47,21,62,60,40.0,44,43.0,36,51,54,60,68,63.0,63,49.0,48,77.0,51,87,25,68,62.0,38.0,39.0,39,49.0,68,67,65.0,11,8,15,13,9,2 ★,2★,Low,High,1 ★,64,30,50,50,66,74,34,67
1,CAM,LM CAM,62,60,44,62,70.0,76,68.0,56,60,70,77,72,83.0,64,79.0,76,62.0,63,47,75,72,26.0,64.0,64.0,60,65.0,19,23,19.0,14,13,9,9,8,4 ★,3★,High,Low,1 ★,74,67,62,74,24,57,12,68
2,GK,GK,7,5,11,21,4.0,6,8.0,8,20,10,28,25,33.0,41,38.0,40,49.0,22,54,6,20,9.0,7.0,26.0,14,31.0,8,9,10.0,56,52,53,53,55,2 ★,1★,Medium,Medium,1 ★,56,52,53,55,26,53,3,54
3,CDM,CB CDM RB,38,31,55,59,32.0,51,34.0,38,47,54,59,58,56.0,48,54.0,48,48.0,60,58,28,61,55.0,33.0,40.0,41,59.0,53,52,54.0,7,5,13,5,6,3 ★,2★,Medium,Medium,1 ★,58,34,47,52,53,59,6,55
4,CDM,CM CDM,57,59,45,78,56.0,71,57.0,51,74,74,68,66,66.0,64,56.0,73,56.0,74,72,62,66,78.0,53.0,62.0,55,63.0,72,68,71.0,12,7,13,15,13,4 ★,2★,Medium,Medium,1 ★,67,62,68,70,69,71,45,70


In [26]:
# cleandata['weight_lbs'] = remove_char(cleandata['weight'],'lbs')
to_drop = ['w/f', 'sm', 'ir']

for e in to_drop:
    cleandata_val[e] = remove_char(cleandata_val[e],'★')
    cleandata_val[e] = cleandata_val[e].astype(int)
    
cleandata_val.head()

,bp,position,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ova
0,CB,CB,47,21,62,60,40.0,44,43.0,36,51,54,60,68,63.0,63,49.0,48,77.0,51,87,25,68,62.0,38.0,39.0,39,49.0,68,67,65.0,11,8,15,13,9,2,2,Low,High,1,64,30,50,50,66,74,34,67
1,CAM,LM CAM,62,60,44,62,70.0,76,68.0,56,60,70,77,72,83.0,64,79.0,76,62.0,63,47,75,72,26.0,64.0,64.0,60,65.0,19,23,19.0,14,13,9,9,8,4,3,High,Low,1,74,67,62,74,24,57,12,68
2,GK,GK,7,5,11,21,4.0,6,8.0,8,20,10,28,25,33.0,41,38.0,40,49.0,22,54,6,20,9.0,7.0,26.0,14,31.0,8,9,10.0,56,52,53,53,55,2,1,Medium,Medium,1,56,52,53,55,26,53,3,54
3,CDM,CB CDM RB,38,31,55,59,32.0,51,34.0,38,47,54,59,58,56.0,48,54.0,48,48.0,60,58,28,61,55.0,33.0,40.0,41,59.0,53,52,54.0,7,5,13,5,6,3,2,Medium,Medium,1,58,34,47,52,53,59,6,55
4,CDM,CM CDM,57,59,45,78,56.0,71,57.0,51,74,74,68,66,66.0,64,56.0,73,56.0,74,72,62,66,78.0,53.0,62.0,55,63.0,72,68,71.0,12,7,13,15,13,4,2,Medium,Medium,1,67,62,68,70,69,71,45,70


In [28]:
def evaluate (d1, d2):
    #     # Printing just 5 results that we know from the Label set(y_train) and 5 predictions to check visualy
    #     # the model working and the scores calculated above
    print('  R2 SCORE: Train', round(r2_score(d1, d2),3))
    print('RMSE SCORE: Train', round(np.sqrt(mean_squared_error(d1, d2)),3))
    print('Prediction Values')
    display(d2)
    print('Real Values')
    display(d1[:5])
    return

_set = cleandata_val.drop(['ova'], axis=1)
 # Making predictions with the new dataset
predictions = lm.predict(_set)
evaluate(data_v['ova'], predictions)

C:\Users\luana\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- a/w
- bp
- d/w
- hits
- position
Feature names seen at fit time, yet now missing:
- a/w_Low
- a/w_Medium
- bp_CB
- bp_CDM
- bp_CF
- ...

  warnings.warn(message, FutureWarning)


ValueError: could not convert string to float: 'CB'

In [17]:
# !git add .

In [18]:
# !git status

In [19]:
# !git commit -m "First lab version 3"

In [20]:
# !git push